# 1　基本構成

Python Flask 檔案通常會命名為 app.py

In [ ]:
from flask import Flask  
app = Flask(__name__) 

`flask` 小寫是 package、`Flask` 大寫是 Class

之後創造一個名稱為 app 的 Flask Object，`__name__` 指定程式執行的位置

In [ ]:
@app.route('/')
def home():
    return "Hello World!"

app.run(port=5000, debug = True)

 用 decorator `@app.route('/')` 來指定下方函式所對應的路徑(URI)
 
 使用 `app.run()` 來執行 Flask 程式，並指定 port 為 5000
 
 `debug = True` 可開啟 Flask 偵錯工具，遇到錯誤會回傳相關 html 網頁
 
 / 指的是根目錄，且函式名稱(home)取什麼都可以
 
 執行後，顯示的 IP位置 127.0.0.1 指的是本機

***

# 2　創造端點(endpoint)

In [ ]:
@app.route('/store', method=['POST'])
def create_store():
    pass

創造一個商店，使用者用 POST 方法送指令給伺服器

若是要用多重方法，可以改為 `method = ['POST', 'GET']`

In [ ]:
@app.route('/store/<string:name>')
def get_store(name):
    pass

`<string:name>` 為任意字串被指派成 `name` 變數並丟入 `get_store` 函式當中

In [ ]:
stores = [
    {
        'name': 'My Store',
        'items': [
            {'name':'my item', 
             'price': 199
            }
        ]
    }
]

可以宣告一個變數來儲存商店資料

`store` 是一個 list，包含許多格式為 dict 的商店

而每個商店 dict 當中的商品項目也是一個 list，包含許多格式為 dict 的單一商品

***

# 3　實作端點功能

In [ ]:
from flask import Flask, jsonify

JSON 格式跟 Python 的 dict 很類似，但本質為跨平台傳輸的純文字格式

亦即需要將 Python 的 dict 轉換成 JSON 格式，此處引入 `jsonify` 以達成此目的

須注意 JSON 格式只支援雙引號　"　而不能使用單引號　'

不過 `jsonify` 會自動處理此問題

In [ ]:
@app.route('/store')
def get_stores():
    
    return jsonify({'stores': stores})

`jsonify` 必須要傳入 dict 格式資料，所以不能直接寫 `jsonify(stores)`

In [ ]:
from flask import Flask, jsonify, request

@app.route('/store' , methods=['POST'])
def create_store():
    
    request_data = request.get_json()

    new_store = {
    'name':request_data['name'],
    'items':[]
    }

    stores.append(new_store)
    return jsonify(new_store)

`request_data = request.get_json()` 即取用用戶傳來的 JSON 資料

`get_json()` 的變形：
- `get_json(force = True)` 不論 header 為何都進行處理
- `get_json(silent = True)` 遇到錯誤不進行回報


取得之後，將該 JSON 資料中標籤為 `name` 的字串取用為新商店的 `name`，預設商品內容為空

最後再將此新創的 dict 格式商店附加在原 `stores` list 後方

In [ ]:
@app.route('/store/<string:name>')
def get_store(name):
    
    for store in stores:
    if store['name'] == name:
          return jsonify(store)

    return jsonify({'message': 'store not found'})

註：`stores` 是格式為 list 的商店清單，而 `store` 是格式為 dict 的單一商店

在函式當中，先觸發第一個 `return` 就不會執行下面的程式碼了，故可以如上述寫

In [ ]:
@app.route('/store/<string:name>/item' , methods=['POST'])
def create_item_in_store(name):

    request_data = request.get_json()

    for store in stores:
        if store['name'] == name:
            new_item = {
                'name': request_data['name'],
                'price': request_data['price']
            }

            store['items'].append(new_item)
            return jsonify(new_item)

    return jsonify ({'message' :'store not found'})

創造物品與創造商店類似

***

# 4　結合 JavaScript

引入 `render_template`

並創建 template 資料夾，將 html 等檔案放入，Flask 會自動尋找及讀取

In [ ]:
from flask import Flask, jsonify, request, render_template

@app.route('/')
def home():
    return render_template('index.html')

如此用戶在存取根目錄時，會自動顯示 index.html 的內容